# Effiziente Programme in Julia

Dieses Jupyter Notebook zeigt anhand einfacher Beispiele, wie mögliche Performance-Probleme in Julia identifiziert und behoben werden können. Um eine sogenannte Zelle in diesem Notebook auszuführen, muss sie zunächst ausgewählt werden (z.B. mit der Maus oder den Pfeiltasten) und kann dann mit `Shift-Enter` gestartet werden.

Beim ersten Mal sollten alle Zellen der Reihe nach ausgeführt werden, da manche Zellen auf Ergebnissen aus vorherigen Zellen aufbauen. Später ist es jedoch möglich - und erwünscht! - selber ein wenig auszuprobieren und den Code ggf. auch zu verändern.

## Performance-Messung mit `@time` und `@benchmark`

Das erste Beispiel behandelt die Funktion `intlog2`, die zu einem Eingabewert den ganzzahligen Logarithmus zur Basis zwei bestimmt. Zunächst werden hier Laufzeitmessungen mit den Makros `@time` und `@benchmark` durchgeführt.

In [ ]:
function intlog2(n)
    r = 0
    while n > 1
        r += 1
        n = n/2
    end
    return r
end

In [ ]:
@time intlog2(1_000_000)

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark intlog2(1_000_000)

## Typstabilitäten erkennen mit `@code_warntype`

Im zweiten Beispiel wird die Funktion `intlog2` mit Hilfe des Makros `@code_warntype` auf Typinstabilitäten untersucht, die anschließend behoben werden.

In [ ]:
@code_warntype intlog2(1_000_000)

In [ ]:
function intlog2_stable(nn)
    r = 0
    n = Float64(nn)
    while n > 1
        r += 1
        n = n/2
    end
    return r
end

In [ ]:
@code_warntype intlog2_stable(1_000_000)

In [ ]:
@benchmark intlog2_stable(1_000_000)

## Typinstabilität und Function Barriers

Im dritten Beispiel geht es um die Funktion `strange_twos`, die eine nicht behebbare Typinstäbilität enthält. Durch eine sogenannte Function Barrier können wir dennoch die Laufzeit deutlich verbessern, indem wir den eigentlichen Rechenkernel in eine neue Funktion `fill_twos` auslagern.

In [ ]:
function strange_twos(n)
  a = Vector{rand(Bool) ? Int : Float64}(undef, n)
  for i in eachindex(a)
    a[i] = 2
  end
  return a
end

In [ ]:
@benchmark strange_twos(10_000)

In [ ]:
function strange_twos_barrier(n)
  a = Vector{rand(Bool) ? Int : Float64}(undef, n)
  fill_twos(a) # Loop-Kernel in neuer Funktion → Function Barrier
  return a
end

function fill_twos(a)
  for i in eachindex(a)
    a[i] = 2
  end
end

In [ ]:
@benchmark strange_twos_barrier(10_000)